In [20]:
import numpy as np
import pandas as pd
import sqlite3
conn = sqlite3.connect('zippedData/im.db')
cur = conn.cursor()

In [2]:
pd.read_sql("""
SELECT name
FROM sqlite_schema
WHERE type='table'
ORDER BY name;""", conn)

,name
0,directors
1,known_for
2,movie_akas
3,movie_basics
4,movie_ratings
5,persons
6,principals
7,writers


,Movie Title,Year,Runtime,Rating,Votes
0,Once Upon a Time ... in Hollywood,2019,159.0,9.7,5600
1,Ekvtime: Man of God,2018,132.0,9.6,2604
2,Aloko Udapadi,2017,113.0,9.5,6509
3,Peranbu,2018,147.0,9.4,9629
4,Dag II,2016,135.0,9.3,100568


In [50]:
df1 = pd.read_sql("""
SELECT * FROM movie_basics MB join movie_ratings MR
    on MB.movie_id = MR.movie_id;""", conn)

In [34]:
df1

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,tt0069049,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",tt0069204,6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",tt0100275,6.5,119
...,...,...,...,...,...,...,...,...,...
73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,tt9913084,6.2,6
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",tt9914286,8.7,136
73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,tt9914642,8.5,8
73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,tt9914942,6.6,5


In [35]:
df1.shape

(73856, 9)

In [36]:
df1['numvotes'].mean()

3523.6621669194105

In [77]:
df2 = pd.read_sql("""
SELECT mb.movie_id,  *
FROM movie_basics MB join movie_ratings MR on MB.movie_id = MR.movie_id
join movie_akas MA on MB.movie_id = MA.movie_id
WHERE numvotes > 5000 AND (region = "US")
Group By MB.movie_id
ORDER BY averagerating desc;""", conn)

In [78]:
df2.head(20)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt7131622,Once Upon a Time ... in Hollywood,Once Upon a Time ... in Hollywood,2019,159.0,"Comedy,Drama",tt7131622,9.7,5600,tt7131622,2,#9,XWW,en,None,informal title,0.0
1,tt7738784,Peranbu,Peranbu,2018,147.0,Drama,tt7738784,9.4,9629,tt7738784,2,Resurrection,IN,en,festival,None,0.0
2,tt5813916,The Mountain II,Dag II,2016,135.0,"Action,Drama,War",tt5813916,9.3,100568,tt5813916,7,The Mountain II,XWW,en,imdbDisplay,None,0.0
3,tt5354160,Aynabaji,Aynabaji,2016,147.0,"Crime,Mystery,Thriller",tt5354160,9.3,18470,tt5354160,2,Mirror Game,BD,en,imdbDisplay,None,0.0
4,tt2170667,Wheels,Wheels,2014,115.0,Drama,tt2170667,9.3,17308,tt2170667,1,Wheels,US,None,None,None,0.0
5,tt5275892,O.J.: Made in America,O.J.: Made in America,2016,467.0,"Biography,Crime,Documentary",tt5275892,8.9,14946,tt5275892,1,O.J.: Made in America,US,None,None,None,0.0
6,tt7060344,Ratsasan,Ratsasan,2018,170.0,"Action,Crime,Thriller",tt7060344,8.8,10518,tt7060344,2,Demon,XWW,en,None,informal literal English title,0.0
7,tt7019842,96,96,2018,158.0,"Drama,Romance",tt7019842,8.8,10903,tt7019842,2,96,US,None,imdbDisplay,None,0.0
8,tt4154796,Avengers: Endgame,Avengers: Endgame,2019,181.0,"Action,Adventure,Sci-Fi",tt4154796,8.8,441135,tt4154796,5,Avengers: Endgame,TH,en,working,None,0.0
9,tt3417422,Drishyam,Drishyam,2013,160.0,"Crime,Drama,Thriller",tt3417422,8.8,24326,tt3417422,2,Visual,XWW,en,None,informal alternative title,0.0


In [90]:
df3 = pd.read_sql("""
SELECT p.primary_name, mb.primary_title, mr.averagerating
FROM persons p join principals pr on 
    p.person_id = pr.person_id
    join movie_basics mb
    on pr.movie_id = mb.movie_id
    join movie_ratings mr on mb.movie_id=mr.movie_id
WHERE mr.numvotes > 3400 
ORDER BY averagerating desc; """, conn)

In [91]:
df3

,primary_name,primary_title,averagerating
0,Brad Pitt,Once Upon a Time ... in Hollywood,9.7
1,Leonardo DiCaprio,Once Upon a Time ... in Hollywood,9.7
2,Quentin Tarantino,Once Upon a Time ... in Hollywood,9.7
3,David Heyman,Once Upon a Time ... in Hollywood,9.7
4,Barbara Ling,Once Upon a Time ... in Hollywood,9.7
...,...,...,...
48222,Arifin Putra,Hanum & Rangga,1.2
48223,Rio Dewanto,Hanum & Rangga,1.2
48224,Benni Setiawan,Hanum & Rangga,1.2
48225,Hanum Salsabiela Rais,Hanum & Rangga,1.2


In [93]:
df4 = pd.read_sql("""
SELECT p.primary_name, mb.primary_title, mr.averagerating
FROM persons p join principals pr on 
    p.person_id = pr.person_id
    join movie_basics mb
    on pr.movie_id = mb.movie_id
    join movie_ratings mr on mb.movie_id=mr.movie_id
WHERE mr.numvotes > 3400 AND primary_name = "Leonardo DiCaprio"
ORDER BY averagerating desc; """, conn)

In [94]:
df4

,primary_name,primary_title,averagerating
0,Leonardo DiCaprio,Once Upon a Time ... in Hollywood,9.7
1,Leonardo DiCaprio,Inception,8.8
2,Leonardo DiCaprio,Django Unchained,8.4
3,Leonardo DiCaprio,Before the Flood,8.3
4,Leonardo DiCaprio,The Wolf of Wall Street,8.2
5,Leonardo DiCaprio,Shutter Island,8.1
6,Leonardo DiCaprio,The Revenant,8.0
7,Leonardo DiCaprio,Hubble,7.7
8,Leonardo DiCaprio,The Great Gatsby,7.3
9,Leonardo DiCaprio,Out of the Furnace,6.8


In [95]:
df4.shape

(16, 3)